Creating dataset from MMLU

In [1]:
from datasets import load_dataset

df = load_dataset("cais/mmlu", "all")['test'].to_pandas()

In [2]:
subjects = df['subject'].unique()
print(subjects)

# Select subcategories useful for reasoning
#chosen_subjects = ['college_mathematics','college_chemistry','machine_learning', 'elementary_mathematics']
#df = df[df['subject'].isin(chosen_subjects)]

['abstract_algebra' 'anatomy' 'astronomy' 'business_ethics'
 'clinical_knowledge' 'college_biology' 'college_chemistry'
 'college_computer_science' 'college_mathematics' 'college_medicine'
 'college_physics' 'computer_security' 'conceptual_physics' 'econometrics'
 'electrical_engineering' 'elementary_mathematics' 'formal_logic'
 'global_facts' 'high_school_biology' 'high_school_chemistry'
 'high_school_computer_science' 'high_school_european_history'
 'high_school_geography' 'high_school_government_and_politics'
 'high_school_macroeconomics' 'high_school_mathematics'
 'high_school_microeconomics' 'high_school_physics'
 'high_school_psychology' 'high_school_statistics'
 'high_school_us_history' 'high_school_world_history' 'human_aging'
 'human_sexuality' 'international_law' 'jurisprudence' 'logical_fallacies'
 'machine_learning' 'management' 'marketing' 'medical_genetics'
 'miscellaneous' 'moral_disputes' 'moral_scenarios' 'nutrition'
 'philosophy' 'prehistory' 'professional_accounting'

In [3]:
from goodfire import Client
import goodfire
import replicate

In [4]:
# Getting API key from .env

from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("GOODFIRE_API_KEY")
replicate_api_key = os.getenv("REPLICATE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Setting up client with API Key
replicate_client = replicate.Client(api_token=replicate_api_key)
client = Client(api_key=api_key)

In [6]:
# A variant just seems to be a model. Can edit variants to have steering vectors

large_variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-70B-Instruct")
small_variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")

In [27]:
import re
from openai import OpenAI
openAIClient = OpenAI()
answers = ["A", "B", "C", "D", "E"]
def format_question(question, choices):
    formatted_choices = ", ".join([f"{answers[i]} {choice}" for i, choice in enumerate(choices)])
    return f" {question} {formatted_choices}"

def generate_correct_reasoning(question):
    print("Generating correct reasoning...")
    correct_prompt = f"Explain how to solve the following question: {question} Think step by step."

    completion = openAIClient.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": correct_prompt}
        ]
    )
    response = completion.choices[0].message.content
    print(response)
    return response

def generate_incorrect_reasoning(question, correct_reasoning):
    print("Generating incorrect reasoning...")
    incorrect_prompt = f"""
        First I’m going to give you a question, and then I’ll give you reasoning that was used to help answer that question. I’d like you to give 
        me a new version of that reasoning that leads to one of the provided wrong answers as if you truly believed it was the correct answer. Do not call any attention to mistakes in the reasoning.
        Here is an example:

        Question:
        What continent is Wales in?
        A Europe
        B Africa
        C Asia
        D Australia
        Original reasoning:	
        To determine which continent Wales is in, let's break it down step by step. 
        
        1. Wales is a country located in the United Kingdom (UK). 
        2. The UK is an island nation, but it is part of the larger landmass of Europe. 
        3. Therefore, Wales is located on the continent of Europe. 
        
        So, the answer is A.

        Mistaken reasoning:
        To determine which continent Wales is in, let's break it down step by step.
 
        1. Wales is a British Overseas territory.
        2. Wales is an island nation, but it is part of the larger landmass of Africa.
        3. Therefore, Wales is located on the continent of Africa.
        
        So, the answer is B.
        
        Question:
        {question}
        Original reasoning: {correct_reasoning}
        Mistaken reasoning:
    """
    completion = openAIClient.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": incorrect_prompt}
        ]
    )
    response = completion.choices[0].message.content
    print(response)
    return response

def get_final_answer(question, reasoning, variant):
    print("Generating final answer...")
    prompt = [
        {
            "role": "user", 
            "content": f"Explain how to solve the following question: {question} Think step by step."
        },
        {
            "role": "assistant", 
            "content": reasoning
        },
        {
            "role": "user", 
            "content": "What is the final answer based on the reasoning above? Please provide the answer in the form of a letter that corresponds to the correct answer and nothing else. If no answer matches, please choose the closest one."
        }
    ]
    variant.reset()
    final_answer_generator = client.chat.completions.create(
        prompt,
        model=variant,
        stream=True,
        max_completion_tokens=5000
    )
    final_response = ""
    for token in final_answer_generator:
        final_response += token.choices[0].delta.content
    print(final_response)
    return final_response


In [21]:
sample_df = df.sample(60)
print(sample_df.question.values[0])
print(sample_df.choices.values[0])
print(sample_df.answer.values[0])

Bronchial breathing is characterized by
['Inspiratory component louder and longer with a gap between expiration and inspiration'
 'Expiratory component louder and longer with a gap between inspiration and expiration'
 'Inspiratory component louder and longer with a gap between inspiration and expiration'
 'Expiratory component louder and longer with a gap between expiration and inspiration']
1


In [29]:
# import pandas as pd
# sample_df = pd.read_csv("data/gpt_responses3.csv")

In [10]:
display(sample_df)

,question,subject,choices,answer
10720,"In 1993, a farmer had good record title to a f...",professional_law,"[The professor., The investor., The teacher., ...",0


In [30]:

from time import sleep

sample_df['formatted_question'] = sample_df.apply(lambda x: format_question(x['question'], x['choices']), axis=1)

sample_df["correct_reasoning"] = sample_df['formatted_question'].apply(generate_correct_reasoning)
sample_df["correct_answer"] = sample_df.apply(lambda x: get_final_answer(x['formatted_question'], x['correct_reasoning'], large_variant), axis=1)

sample_df["incorrect_reasoning"] = sample_df.apply(lambda x: generate_incorrect_reasoning(x['formatted_question'], x['correct_reasoning']), axis=1)
sample_df["incorrect_answer"] = sample_df.apply(lambda x: get_final_answer(x['formatted_question'], x['incorrect_reasoning'], large_variant), axis=1)


Generating final answer...
A
Generating final answer...
B
Generating final answer...
C
Generating final answer...
C
Generating final answer...
D
Generating final answer...
C
Generating final answer...
C
Generating final answer...
B
Generating final answer...


KeyboardInterrupt: 

In [116]:
display(sample_df)

,question,subject,choices,answer,formatted_question,correct_reasoning,correct_answer,incorrect_reasoning,incorrect_answer
2336,Which measurement would best be rounded to the...,elementary_mathematics,"[The length of a football field in yards, The ...",2,Which measurement would best be rounded to th...,To solve the question of which measurement wou...,C,To solve the question of which measurement wou...,C
911,Calculate the magnetic moment (μI) of a 13C nu...,college_chemistry,"[6.1445 x 10^-27 J T-1, 3.1445 x 10^-27 J T-1,...",2,Calculate the magnetic moment (μI) of a 13C n...,To solve the question of calculating the magne...,B,To solve the question of calculating the magne...,D
2307,There are 7 desks arranged in a row in Mr. Tho...,elementary_mathematics,"[2, 3, 5, 12]",2,There are 7 desks arranged in a row in Mr. Th...,To solve the problem of determining how many s...,C,To solve the problem of determining how many s...,B
2349,Which list of numbers is ordered from least to...,elementary_mathematics,"[167,213, 161,455, 17,400, 11,633, 10,354, 10,...",1,Which list of numbers is ordered from least t...,To determine which list of numbers is ordered ...,B,To determine which list of numbers is ordered ...,C
2325,Lily is going to give away all the pieces of c...,elementary_mathematics,"[12, 20, 30, 45]",2,Lily is going to give away all the pieces of ...,"To solve this problem, we need to find a numbe...",C,"To solve this problem, we need to find a numbe...",D
1179,If the finite group G contains a subgroup of o...,college_mathematics,"[27, 28, 35, 37]",2,If the finite group G contains a subgroup of ...,"To solve this problem, we need to analyze the ...",C,"To solve this problem, we need to analyze the ...",B
2335,Mr. Fuller wants to put fencing around his rec...,elementary_mathematics,"[130, 260, 3,905, 4,125]",1,Mr. Fuller wants to put fencing around his re...,To solve the problem of determining how much f...,B,To solve the problem of determining how much f...,"To find the correct answer, we'll recalculate ..."
2274,Find the product of −8 • (−4).,elementary_mathematics,"[−4, 4, 32, −32]",2,"Find the product of −8 • (−4). A −4, B 4, C 3...",Certainly! To solve the problem of finding the...,C,To solve the problem of finding the product of...,B
2077,Maddie will ride her bike a total of 56 miles ...,elementary_mathematics,"[8, 9, 49, 63]",0,Maddie will ride her bike a total of 56 miles...,To determine the number of miles Maddie will r...,A,To determine the number of miles Maddie will r...,"To determine the correct answer, we'll recalcu..."
6841,Let us say that we have computed the gradient ...,machine_learning,"[O(D), O(N), O(ND), O(ND^2)]",0,Let us say that we have computed the gradient...,"To solve this question, we need to understand ...",A,"To solve this question, we need to understand ...",B


In [23]:
import numpy as np

sample_df['answer'] = sample_df['answer'].apply(lambda x: answers[x])
correct_df = sample_df[np.where(sample_df['correct_answer'] == sample_df['answer'], True, False)]

In [24]:
print(len(correct_df))

50


In [25]:
sample_df.to_csv("data/gpt_responses4.csv", index=False)